<h1 style="text-align:center;">ESCUELA POLITÉCNICA NACIONAL</h1>
<h2 style="text-align:center;">FACULTAD DE INGENIERÍA DE SISTEMAS</h2>
<h3 style="text-align:center;">INGENIERÍA DE COMPUTACIÓN</h3>

**PERÍODO ACADÉMICO:** 2025-B

**ASIGNATURA:** ICCD412 Métodos Numéricos  

**GRUPO:** GR2  

**TIPO DE INSTRUMENTO:** Proyecto bimestral  

**FECHA DE ENTREGA LÍMITE:**   

**ALUMNOS:** López Alejandro, Velastegui Joel, Guerra Melissa

# LIBRERÍAS

##### Se utilizarán las librerás pandas y numpy, para poder trabajar con la base de datos y aplicar los métodos numéricos que vamos a emplear.

In [ ]:
import numpy
import pandas